## Importation des données

In [16]:
import pandas as pd

logs = pd.read_csv("FW_3.csv")

logs.head()

,datetime,ipsrc,ipdst,dstport,proto,action,policyid
0,2024-03-11 11:54:04,192.168.182.1,192.168.182.130,58918.0,TCP,Deny,34.0
1,2024-03-11 11:54:04,192.168.182.1,192.168.182.130,58918.0,TCP,Deny,34.0
2,2024-03-11 11:54:29,159.84.181.226,192.168.182.130,34106.0,TCP,Deny,34.0
3,2024-03-11 11:54:29,192.168.182.1,192.168.182.130,58918.0,TCP,Deny,34.0
4,2024-03-11 11:54:29,192.168.182.1,192.168.182.130,58918.0,TCP,Deny,34.0


In [17]:
print(logs.shape)
print(logs.info())

(2905, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2905 entries, 0 to 2904
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  2905 non-null   object 
 1   ipsrc     2903 non-null   object 
 2   ipdst     2903 non-null   object 
 3   dstport   2903 non-null   float64
 4   proto     2903 non-null   object 
 5   action    2903 non-null   object 
 6   policyid  2903 non-null   float64
dtypes: float64(2), object(5)
memory usage: 159.0+ KB
None


In [18]:
# ligne vide à supprimer

print(sum(logs['dstport'].isna()))
print(logs[logs['dstport'].isna()])
logs = logs.drop([513,514])
print(logs.shape)

2
                datetime ipsrc ipdst  dstport proto action  policyid
513  2024-03-11 19:19:49   NaN   NaN      NaN   NaN    NaN       NaN
514  2024-03-11 19:39:49   NaN   NaN      NaN   NaN    NaN       NaN
(2903, 7)


In [19]:
logs['dstport'] = logs['dstport'].astype(int).astype(str)
logs['policyid'] = logs['policyid'].astype(int).astype(str)
logs.head()

,datetime,ipsrc,ipdst,dstport,proto,action,policyid
0,2024-03-11 11:54:04,192.168.182.1,192.168.182.130,58918,TCP,Deny,34
1,2024-03-11 11:54:04,192.168.182.1,192.168.182.130,58918,TCP,Deny,34
2,2024-03-11 11:54:29,159.84.181.226,192.168.182.130,34106,TCP,Deny,34
3,2024-03-11 11:54:29,192.168.182.1,192.168.182.130,58918,TCP,Deny,34
4,2024-03-11 11:54:29,192.168.182.1,192.168.182.130,58918,TCP,Deny,34


In [25]:
ip_bad = ["10.0.0.1", "192.168.4.1", "172.156.2.26", "192.168.1.0", "10.10.1.101", "89.89.56.2", "28.12.15.20",
"10.172.11.2", "194.25.56.2", "172.5.2.8"]

In [42]:
print(f"Nombre d'ip malveillantes : {sum(logs['ipsrc'].isin(ip_bad))}")
logs['is_ip_bad'] = logs['ipsrc'].isin(ip_bad)

Nombre d'ip malveillantes : 25


In [46]:
logs['is_ip_bad'].value_counts() 

is_ip_bad
False    2878
True       25
Name: count, dtype: int64

## Serie temporelle

In [72]:
# Convertir la colonne 'datetime' en format datetime
logs['datetime'] = pd.to_datetime(logs['datetime'])

# Créer une nouvelle colonne 'Date par jour'
logs['Jour'] = logs['datetime'].dt.date

# Grouper par 'Jour' et compter le nombre d'occurrences de 'is_ip_bad=True'
ts = logs[logs['is_ip_bad']].groupby('Jour').size().reset_index(name='Number of ip_bad')

# Afficher le résultat
ts

,Jour,Number of ip_bad
0,2024-03-12,25


Pas de serie temporelle les intrusions malveillantes sont toutes le même jour.

## Classification

In [117]:
import pandas as pd
logs = pd.read_csv("attacks.csv")

In [113]:
# # random over sampling

# majority_class = logs[logs['is_ip_bad'] == False]
# minority_class = logs[logs['is_ip_bad'] == True]

# # Dupliquer aléatoirement des exemples de la classe minoritaire
# oversampled_minority = minority_class.sample(n=len(majority_class), replace=True, random_state=42)

# # Concaténer les classes majoritaires et minoritaires suréchantillonnées
# logs = pd.concat([majority_class, oversampled_minority])

In [118]:
from sklearn.model_selection import train_test_split

Xcols = ["ipsrc", "ipdst", "dstport", "proto", "action", "policyid"]
X = logs[Xcols]
y = logs.is_ip_bad

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Convertir les variables catégorielles en numériques avec LabelEncoder
label_encoder = LabelEncoder()
X_encoded = X.apply(label_encoder.fit_transform)

# Utiliser l'encodage One-Hot pour traiter les variables catégorielles
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' pour éviter la colinéarité
X_encoded_onehot = onehot_encoder.fit_transform(X_encoded)

X_train, X_test, y_train, y_test = train_test_split(X_encoded_onehot, y, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2032, 347)
(871, 347)
(2032,)
(871,)


In [119]:
# Oversampling avec méthoe SMOTE
from imblearn.over_sampling import SMOTENC

# Appliquer SMOTE sur l'ensemble d'entraînement
smote = SMOTENC(categorical_features=[0, 1, 3, 4], random_state=42) #dstport et policyid en tant que numerical_feature
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(X_train_resampled.shape)
print(y_train_resampled.shape)

(4032, 347)
(4032,)


In [120]:
y_train_resampled.value_counts()

is_ip_bad
False    2016
True     2016
Name: count, dtype: int64

In [108]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, auc


# Entraîner un modèle de classification (par exemple, Random Forest)
model = RandomForestClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Afficher le rapport de classification
print(f"Accuracy : {accuracy_score(y_test, y_pred)}")
print(f"Precision : {precision_score(y_test, y_pred)}")
print(f"Recall : {recall_score(y_test, y_pred)}")
print(f"F1 score : {f1_score(y_test, y_pred)}")
print(f"Matrice de confusion : \n {confusion_matrix(y_test, y_pred)}")


Accuracy : 0.9931113662456946
Precision : 1.0
Recall : 0.3333333333333333
F1 score : 0.5
Matrice de confusion : 
 [[862   0]
 [  6   3]]


In [123]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

# Initialiser le modèle Random Forest
model = RandomForestClassifier(random_state=42)

# Définir la grille des hyperparamètres à rechercher
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialiser l'objet GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='recall', n_jobs=-1)

# Exécuter la recherche sur la grille
grid_search.fit(X_encoded_onehot, y)

# Afficher les meilleurs hyperparamètres et le meilleur score
print("Meilleurs hyperparamètres :", grid_search.best_params_)
print("Meilleur score de rappel (recall) :", grid_search.best_score_)

Meilleurs hyperparamètres : {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Meilleur score de rappel (recall) : 0.6399999999999999


In [126]:
import pickle

# Sauvegarder le modèle en utilisant pickle
best_model = grid_search.best_estimator_
with open('random_forest_model.pkl', 'wb') as model_file:
    pickle.dump(best_model, model_file)